<a href="https://colab.research.google.com/github/sofiaaic/PrediccionTendenciasPoliticasChile/blob/joaquin/notebooks/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Acá dejaremos el codigo para extraer los datos RAW.

In [2]:
# (Opcional) instala motores para Excel si hace falta
!pip -q install openpyxl


In [ ]:
# 1) Montar Drive
from google.colab import drive
drive.mount('/content/drive')  # autoriza con tu cuenta


In [ ]:

# 2) Configurar ruta de carpeta en tu Drive
#    Ejemplo: "Mi unidad/Proyectos/Excels"
import os, glob
import pandas as pd

CARPETA = '/content/drive/MyDrive/data_votantes_chile'  # ajusta a tu ruta


# 3) Buscar archivos .xlsx (evitar temporales ~$.xlsx)
rutas = [p for p in glob.glob(os.path.join(CARPETA, '**', '*.xlsx'), recursive=True)
         if not os.path.basename(p).startswith('~$')]

print(f'Encontrados {len(rutas)} archivos:')
for r in rutas: print('-', os.path.relpath(r, CARPETA))

# 4) Leer y concatenar (elige la hoja, columnas, tipos, etc.)
dfs = []
for ruta in rutas:
    # Personaliza: sheet_name, usecols, dtype, skiprows…
    df = pd.read_excel(
        ruta,
        sheet_name=0,        # o el nombre: 'Hoja1'
        engine='openpyxl',
        # usecols='A:D',     # ejemplo para limitar columnas
        # dtype={'RUT': str} # ejemplo para forzar tipos
    )
    df['__archivo_origen'] = os.path.basename(ruta)  # traza
    dfs.append(df)

df_total = pd.concat(dfs, ignore_index=True)
df_total.head()
